![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [182]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as MSE

# Import any additional modules and start coding below
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso

In [183]:
df = pd.read_csv("rental_info.csv")
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


### Find dtypes for columns and convert the date columns to pandas datetime

In [184]:
df.dtypes

df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])
df.dtypes

rental_date         datetime64[ns, UTC]
return_date         datetime64[ns, UTC]
amount                          float64
release_year                    float64
rental_rate                     float64
length                          float64
replacement_cost                float64
special_features                 object
NC-17                             int64
PG                                int64
PG-13                             int64
R                                 int64
amount_2                        float64
length_2                        float64
rental_rate_2                   float64
dtype: object

### Subtract the dates to get days using dt.days

In [185]:
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days
df.head(20)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4
5,2005-05-29 16:51:44+00:00,2005-06-01 21:43:44+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
6,2005-06-17 19:42:42+00:00,2005-06-22 20:39:42+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,5
7,2005-07-09 18:23:46+00:00,2005-07-13 19:04:46+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4
8,2005-07-27 13:16:28+00:00,2005-07-28 13:40:28+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
9,2005-08-21 13:53:52+00:00,2005-08-25 09:03:52+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3


### Use masking to return ones and zeros for the desired columns

In [186]:
df['deleted_scenes'] = df['special_features'].str.contains("Deleted Scenes").astype(int)
df['behind_the_scenes'] = df['special_features'].str.contains("Behind the Scenes").astype(int)
df.head(20)
# df['behind_the_scenes'].unique()
df['deleted_scenes'].value_counts()

0    7973
1    7888
Name: deleted_scenes, dtype: int64

### Use np.where to achieve the same result

In [187]:
# df['deleted_scenes'] = np.where(df['special_features'].str.contains('Deleted Scenes'), 1, 0)
# df['behind_the_scenes'] = np.where(df['special_features'].str.contains('Behind the Scenes'), 1, 0)

# df['deleted_scenes'].value_counts()

### Removing leaky features

In [188]:
df.drop(columns=["rental_date", "return_date", "special_features"], inplace=True)
df.head()
df.shape

(15861, 15)

### Splitting data

In [189]:
X = df.drop("rental_length_days", axis=1)
y = df['rental_length_days']
X.shape
print(y.shape)

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
print(X_train.shape)
print(y_train.shape)

(12688, 14)
(12688,)


### Training Naive models

In [191]:
lasso = Lasso(max_iter=1500, random_state=9)
rf = RandomForestRegressor(max_features=0.8, max_depth=4, n_estimators=300, n_jobs=-1, oob_score=True, random_state=9)
gb = GradientBoostingRegressor(max_features=0.8, max_depth=4, n_estimators=200, random_state=9)
# gb.get_params()

In [192]:
classifiers = [("lasso",lasso), ("randomforest", rf), ('gradientboosting', gb)]
mean_error = {}
for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    mean_error[clf_name] = np.sqrt(MSE(y_test, y_pred))

In [193]:
sorted(mean_error.items(), key= lambda item: item[1], reverse=True)

[('lasso', 1.9508173695313487),
 ('randomforest', 1.6903732490237193),
 ('gradientboosting', 1.48279997087639)]

### Since GradientBoosting is looking good, use GridSearch to find best parameters

In [ ]:
param_dict ={"max_features": np.linspace(0.1, 1.0, 5), 
             "max_depth": range(2,6), 
             "n_estimators": [100, 250, 300], 
             "random_state": [3, 4, 9],
            }
gb_model = GradientBoostingRegressor()
# gb_model.get_params()
grid = GridSearchCV(estimator=gb_model, param_grid=param_dict, cv=3, n_jobs=-1)

grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_estimator_)
# grid.get_params()

In [194]:
best_model = grid.best_estimator_
best_model.fit(X_train, y_train)
pred = best_model.predict(X_test)


best_mse = MSE(y_test, pred)
print(best_mse)
np.sqrt(best_mse)

1.9836854361637055


1.4084336818479262

In [195]:
features_imp = best_model.feature_importances_
features_imp

array([0.28405756, 0.01571459, 0.08752235, 0.03221303, 0.02892342,
       0.00345346, 0.00226886, 0.00288118, 0.00324823, 0.3906702 ,
       0.0334554 , 0.10630097, 0.00455868, 0.00473208])

In [196]:
feature_importance = pd.Series(features_imp, index=X_train.columns)
feature_importance.sort_values(ascending=False, inplace=True)
feature_importance.head()

amount_2         0.390670
amount           0.284058
rental_rate_2    0.106301
rental_rate      0.087522
length_2         0.033455
dtype: float64

In [204]:
pred_series = pd.Series(y_pred)
pred_series.name = 'predictions'
y_actual = pd.Series(y_test).reset_index(drop=True)
view_preds = pd.concat([y_actual, pred_series.astype(int)], axis=1)
view_preds

,rental_length_days,predictions
0,8,7
1,1,2
2,6,2
3,9,7
4,5,2
...,...,...
3168,9,6
3169,9,7
3170,5,2
3171,3,2
